# <div align="center"> CODEX: Coverage of Data Explorer </div>
The Coverage of Data Explorer (CODEX) tool is a Python package that implements data coverage metrics and algorithms for artificial intelligence (AI)/machine learning (ML) test and evaluation (T&E) applications. CODEX’s metrics are based on the theory of combinatorial testing (CT) adapted from software testing to AI/ML T&E with a data-centric focus. 

As a data assurance tool, CODEX requires datasets for all functionalities. Some functionalities additionally require ML training algorithms and a mechanism for automatically training models and evaluating them on a test set (a test harness). CODEX provides functionalities such as:

- Evaluation of the coverage of a defined universe by a dataset, which may be used for applications such as selecting a model for deployment domain
- Between dataset coverage, which may be used for applications such as computing the difference between a test and training set to generate representative and challenging test sets
- Systematic inclusion/exclusion framework (SIE), which is used for identification of critical metadata factors
- Data splitting algorithms, which may be used to construct balanced data sets for training or testing. The SIE framework uses the data splitting algorithm for constructing a balanced universal test set. In the future, this framework could use the same algorithm to produce training sets that are not only covering but also balanced. 
- Prioritization of samples to best cover a space, which may be used to support high information gain in resource restricted scenarios such as labeling and retraining

# Quickstart Tutorial
- CODEX can be imported as a Python library with `import codex` or directly from running `python codex/codex.py` as a main entry point.

## Prerequisites
- CODEX, developed for Python 3, requires multiple packages including `pandas`, `numpy`, and `matplotlib`.
- Dependencies for CODEX can be installed upon installation as a library with
`python -m pip install ../`
- If being ran directly, dependenceis for CODEX can be installed with
`python -m pip install ../requirements.txt`



In [ ]:
!python -m pip install ../
!python -m pip install -r ../requirements.txt
!python -m pip install directory_tree

In [ ]:
import codex.codex as codex
from codex.modules import output
import os
import json
import pandas as pd
from PIL import Image

### Learning Objectives
After following this notebook, you should be able to:
- Understand what is required for a codex environment and how to set one up.
- Fill out a CODEX config file and run experiment via command line OR as a module.
- How to construct a universe for CODEX.
- View format requirements for datasets, splits, and performance files.
- Find and view the outputs of an experiment.

## CODEX Setup

### CODEX directory
Dedicating a specific directory for applying CODEX to a specific dataset is encouraged. The concept of a 'CODEX directory' is for the sake of preservation of input materials for experiments and other artifacts. While elements can be passed to CODEX in a piecewise fashion, utilizing a CODEX directory helps to keep materials organized.

In [ ]:
codex_dir = codex.setup_new_codex_env('new', templates=True, tutorial=True)

At most, a proper CODEX directory contains the following:
- Config files: .json files specifying file locations
- Binning files: .txt files explicitly listing features and values while binning continuous feature-values.
- Performance files: .json files containing overall as well as per-sample performance.
- Split files: .json files containing sample ID's reflecting samples ID's from the dataset for each split.

### Dataset
CODEX is a tool that evaluates a dataset for vairious applications in machine learning.

Because of the combinatorial testing-based framework of CODEX, CODEX requires a tabular dataset as its input data, that is, a table/matrix format of data with rows of samples and columns of features, which hold values for features.

In [ ]:
display(pd.read_csv(os.path.join(codex_dir, 'data', 'dataset_rareplanes_lightweight_EX.csv')))

### Universe Construction
Combinatorial coverage metrics are computed with respect to a defined universe describing the input dataset. Because of this combinatorial approach, the defined universe must be constructed with discrete values, where continuous features in the data are discretized. Providing the universe for the experiment can be done one of three ways.

1) Learning from dataset: Upon reading in the dataset, the universe is constructed by treating each appearing unique value for a feature as a level in the universe. Note that for this method, discretizing continouous features by uniqueness of values can result in an infinite amount levels for a continuous feature.
2) Using a previous universe: Universes from a CODEX experiment can be saved. Whether user-written or from a previous universe, providing a directory or file path to the universe file through the input file, the universe for one experiment can applied to any other. 
3) Binning file: A separate text file known as the binning file specifying the binning scheme of features and values can be provided to construct the universe.
- A major advantage of this method lies in the semantic meaning the user can apply in the binning schemes. Continuous variables can have customized bins depending on the nature of the feature, and categorical variables can be reordered for more meainingful order (e.g., ordering 'Spring', 'Summer', 'Fall', 'Winter', the feature 'Season' rather than the order it appears in the dataset). 
    
For methods 2) and 3), these methods of universe construction are valid so long as:
- No feature or value exists in the dataset that does not exist in  while option `use_augmented_universe` is off. If `use_augmented_universe` is on, features and/or values expand the current universe.
- Bins for continuous features at minimum cover the range of values that appear in dataset, meaning maximum and minimum are included.



#### Format
- Binning file (`.txt`)
    - Features are specified on each line, separated from its values with a colon (':').
    - Values are specified following, separated from its values with semicolons (';').
- Universe file (1-layer `dict`/`.json`)
    - Under a `features` key, a list of feature names.
    - Under a `values`, a 2D list containing values corresponding to each feature.

#### Previous universe

In [ ]:
with open(os.path.join('..', 'resources', 'templates', 'universe_TEMPLATE.json')) as f:
    predefined_universe = json.load(f)
    display(predefined_universe)

#### Binning file

In [ ]:
binning = os.path.join('..', 'resources', 'templates', 'binning_TEMPLATE.txt')
dataset_unbinned = os.path.join('..', 'resources', 'templates', 'dataset_EXAMPLE.csv')

with open(os.path.join('..', 'resources', 'templates', 'binning_TEMPLATE.txt')) as f:
    print(f.read())

## Running CODEX: Dataset Evaluation
To observe how CODEX uses all these elements to run an experiment, we will run CODEX `dataset_split_evaluation` on an example dataset.

### Dataset: RarePlanes
RarePlanes is an open source dataset consisting of real and synthetic images of planes overhead. Of the real dataset, 253 satellite images are distributed among 8,525 image tiles. RarePlanes also includes a metadata table for each of the 253 images, which can be used for CODEX.

Below is an example of one such tile, along with its associated metadata as it would appear in the `.csv` dataset.


In [ ]:
rareplanes_df = pd.read_csv(os.path.join(codex_dir, 'data', 'dataset_rareplanes_lightweight_EX.csv'), index_col = 'image_tile_id')

display(rareplanes_df.loc['84_1040010049B46C00_tile_202'])
display(Image.open(os.path.join('..', 'resources', '84_1040010049B46C00_tile_202.png')))

In [ ]:
with open(os.path.join(codex_dir, 'configs', 'configs_rareplanes_EX.json')) as f:
    input_file = json.load(f)

input_file['codex_directory'] = codex_dir
input_file['output_directory'] = codex_dir
input_file['bin_directory'] = os.path.join(codex_dir, 'binning')
input_file['data_directory'] = os.path.join(codex_dir, 'data')
display(input_file)

In [ ]:
codex.run(input_file, verbose='1')

## Running CODEX with extra requirements: Dataset Split Evaluation
Certain modes of CODEX require extra components for the analysis. `dataset_split_evaluation` computes set difference combinatorial coverage (SDCC) between two portions of a single dataset with respect to a defined universe, which can be used describe the relationship between splits of a dataset, like test and train.

Because `dataset_split_evaluation` is examining different portions of a dataset, this mode requires an extra element, the split file which specifies which samples belong to which split. The sample IDs in this split file are as they appear in the dataset.

In [ ]:
train_ids = []
val_ids = []
test_ids = []

split = {'train': train_ids,
         'validation': val_ids,
         'test': test_ids}
codex.output.output_json_readable(split, write_json=True, file_path=os.path.join(codex_dir, 'splits', 'split_example.json'))

input['split_dir'] = 'splits'
input['split_file'] = 'split_exmaple.json'

In [ ]:
codex.run(input, verbose='1')